# Проект "Рекомендация тарифов"

## Открытие и изучение файла с данными

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [11]:
df.shape

(3214, 5)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [13]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Выводы:**

- подключили все необходимые функции библиотеки sklearn;
- загрузили данные с 3214 объектами и 5 признаками;
- поскольку подбираем тарифный план - целевой признак находится в столбце "is_ultra".

## Разделение исходных данных на обучающую, валидационную и тестовую выборки

Разделим датафрейм на столбцы с целевым признаком - 'is_ultra' и столбцы с остальными признаками:

In [14]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Далее разделим полученные массивы на на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Сначала выделим тестовую выборку:

In [15]:
features_train, features_test, target_train, target_test = (train_test_split(features, 
                                                            target, test_size=0.2, random_state=12345))

Чтобы сохранить соотношение 3:1:1 для получения валидационной выборки надо взять одну четвертую часть от тренировочной выборки выше:

In [16]:
features_train, features_valid, target_train, target_valid = \
train_test_split(features_train, target_train, test_size=0.25, random_state=12345)

Размеры выборок:

In [17]:
features_test.shape[0] #тестовая выборка

643

In [18]:
features_train.shape[0] #тренировочная выборка

1928

In [19]:
features_valid.shape[0] #валидационная выборка

643

**Выводы:**

- скрытой тестовой выборки не было по условию, поэтому ипользовал разбиение в соотношении 3:1:1;
- последовательно использовал функцию train_test_split сначала для формирования тестовой и тренировочной выборок, а затем для формирования валидационной выборки из тренировочной с изменением процентных долей;
- получил длину тестовой и валидационной выборок 643, длину тренировочной выборки - 1928.

## Исследование качества разных моделей

Рассмотрим три изученные модели: решающее дерево, случайный лес и логистическая регрессия. Для этого вначале проекта подключим соответствующие модели из библиотеки sklearn.

**6.1 Решающее дерево**

Инициализируем модель решающего дерева:

In [20]:
model = DecisionTreeClassifier(random_state=12345)

Обучим модель на тренировочных данных:

In [21]:
model.fit(features_train, target_train)

DecisionTreeClassifier(random_state=12345)

In [23]:
model.get_depth()

28

Получим предсказания по валидационной выборке:

In [24]:
predictions = model.predict(features_valid)

Считаем качество accuracy, подключив перед этим соответствующую функцию из библиотеки sklearn:

In [25]:
accuracy_score(target_valid, predictions)

0.7122861586314152

Попробуем подобрать параметры 

In [27]:
best_model = None
best_result = 0
for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # инициируем модель с заданной глубиной дерева
	model.fit(features_train, target_train) # обучаем модель
	predictions = model.predict(features_valid) # получаем предсказания модели
	result = accuracy_score(target_valid, predictions) # считаем качество модели
	if result > best_result:
		best_model = model
		best_result = result
        
print("Accuracy лучшей модели:", best_result)
print("Лучшая модель:", best_model)

Accuracy лучшей модели: 0.7651632970451011
Лучшая модель: DecisionTreeClassifier(max_depth=3, random_state=12345)


**6.2 Cлучайный лес**

Будем сразу перебирать варианты с разными гиперпараметрами (n_estimators и max_depth): 

In [28]:
best_model = None
best_result = 0

for est in (10, 51, 3):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) # обучаем модель
        predictions = model.predict(features_valid) # получаем предсказания модели
        result = accuracy_score(target_valid, predictions) # считаем качество модели
        if result > best_result:
            best_model = model
            best_result = result
        
print("Accuracy лучшей модели:", best_result)
print("Лучшая модель:", best_model)

Accuracy лучшей модели: 0.7962674961119751
Лучшая модель: RandomForestClassifier(max_depth=9, n_estimators=3, random_state=12345)


**6.3 Логистическая регрессия**

Инициализируем модель логистической регрессии с типовыми параметрами из теоретической части курса:

In [29]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)

Обучим модель на тренировочных данных:

In [30]:
model.fit(features_train, target_train)

LogisticRegression(max_iter=1000, random_state=12345)

Получим предсказания по валидационной выборке:

In [31]:
predictions = model.predict(features_valid)

Считаем качество accuracy:

In [32]:
accuracy_score(target_valid, predictions)

0.7262830482115086

Выводы:
- проанализировано три типа модели: решающее дерево с разной глубиной дерева (лучший результат правильности - 0.76 с глубиной 3), случайный лес с разными гиперпараметрами (лучший результат правильности - 0.79 с глубиной 9 и числом оценщиков 3) и логистическая регрессия (0.72);
- для дальнейшей проверки качества на тестовой выборке выбираем модель с максимальным accuracy:
Accuracy лучшей модели: 0.7962674961119751
Лучшая модель: RandomForestClassifier(max_depth=9, n_estimators=3, random_state=12345)

## Проверка качества модели на тестовой выборке

Проверим качество выбранной модели с лучшими параметрами на тестовой выборке:

Инициализируем модель:

In [27]:
model = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=3)

Обучим модель на тренировочных данных:

In [28]:
model.fit(features_train, target_train)

RandomForestClassifier(max_depth=9, n_estimators=3, random_state=12345)

Получим предсказания по тестовой выборке:

In [29]:
predictions = model.predict(features_test)

Считаем качество accuracy:

In [30]:
accuracy_score(target_test, predictions)

0.7931570762052877

Выводы:
На тестовой выборке проверено качество модели случайный лес: RandomForestClassifier(max_depth=9, n_estimators=3, random_state=12345). Значение accuracy = 0.79, что удовлетворяет поставленным требованиям.

## Проверка модели на адекватность

В основе проверки на адекватность лежит очень простая идея. Модель считается адекватной, если способна "побить" любой "наивный" алгоритм, например предсказывающий случайный или постояный ответ.
Для нашего проекта самую высокую точность дает "наивный" алгоритм, всегда предсказывающий самый популярный ответ (у нас это 0). Проверить точность такого подхода можно вручную или используя DummyClassifier.

Найдем доли тарифных планов в датасете. Доля тарифа ultra:

In [58]:
target_train.sum() / len(target_train)

0.30549792531120334

Значит доля тарифа смарт:

In [59]:
1 - target_train.sum() / len(target_train)

0.6945020746887967

значит если модель будет всегда предсказывать тариф смарт, то accuracy составит 0.69.

In [33]:
from sklearn.dummy import DummyClassifier

In [64]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train) # значение accuracy

0.6945020746887967

**Выводы:**

Accuracy для выбранной модели превосходит accuracy для случайной модели: 0.79 > 0.69. Следовательно, полученная модель адекватна.

## Общий вывод по исследованию

Построена модель для задачи классификации (выбор подходящего тарифного плана) с максимально большим значением accuracy:

Accuracy лучшей модели: 0.7962674961119751
Лучшая модель: RandomForestClassifier(max_depth=9, n_estimators=3, random_state=12345)

Адекватность модели проверена путем сравнения с наивной моделью, предсказывающей наиболее популярный тариф смарт(0.79 > 0.69).